In [ ]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import numpy as np

client = RemoteAPIClient()

sim = client.getObject("sim")

robotHandle = sim.getObject("/Manta")
steerHandle = sim.getObject("/Manta/steer_joint")
motorHandle = sim.getObject("/Manta/motor_joint")

motorTorque = 60
motorVelocity = 12
sim.setJointForce(motorHandle, motorTorque)
sim.setJointTargetVelocity(motorHandle, motorVelocity)

robotPos = sim.getObjectPosition(robotHandle, -1)

maxSteer = np.deg2rad(5)

sim.startSimulation()

while (t := sim.getSimulationTime()) < 20:

    dt = sim.getSimulationTimeStep()
    robotPos = sim.getObjectPosition(robotHandle, -1)
    
    # Entrada de Controle
    u =  maxSteer
    if robotPos[1] > 0:
        u = -maxSteer

    sim.setJointTargetPosition(steerHandle, u)

sim.stopSimulation()

In [85]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import numpy as np

client = RemoteAPIClient()

sim = client.getObject("sim")

robotHandle = sim.getObject("/Manta")
steerHandle = sim.getObject("/Manta/steer_joint")
motorHandle = sim.getObject("/Manta/motor_joint")

motorTorque = 60
motorVelocity = 12
sim.setJointForce(motorHandle, motorTorque)
sim.setJointTargetVelocity(motorHandle, motorVelocity)

robotPos = sim.getObjectPosition(robotHandle, -1)

maxSteer = np.deg2rad(20)

setPoint = 0

error = 0
lastError = 0
cumError = 0

sim.startSimulation()

while (t := sim.getSimulationTime()) < 20:

    dt = sim.getSimulationTimeStep()
    robotPos = sim.getObjectPosition(robotHandle, -1)
    
    # Error        
    error = setPoint - robotPos[1]
    dError = (error - lastError) / dt
    cumError += error * dt
    
    # kp = .095 kd = .95 ki = .0001

    # Proporcional
    kp = .095
    up = kp*error
        
    # Derivativo
    kd = .95
    ud = kd*dError
        
    # Integral
    ki = .0001
    ui = ki*cumError

    # Controller
    u = up + ud + ui 
    
    # Limitando o valor para +/- max
    u = max(min(u, maxSteer), -maxSteer)
    
    sim.setJointTargetPosition(steerHandle, u)
    
    lastError = error    

sim.stopSimulation()

In [130]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import numpy as np

# Normalize angle to the range [-pi,pi)
def normalizeAngle(angle):
    return np.mod(angle+np.pi, 2*np.pi) - np.pi

client = RemoteAPIClient()

sim = client.getObject("sim")

robotHandle = sim.getObject('/PioneerP3DX')
leftMotor = sim.getObject('/PioneerP3DX/leftMotor')
rightMotor = sim.getObject('/PioneerP3DX/rightMotor')

qgoal = np.array([0, 2, np.deg2rad(45)])

goalFrame = sim.getObject('/Goal')
sim.setObjectPosition(goalFrame, [qgoal[0], qgoal[1], 0.2])
sim.setObjectOrientation(goalFrame, [0,0,qgoal[2]])

# Dados do robo P3DX
L = 0.381
r = 0.0975
maxv = 1.0
maxw = np.deg2rad(45)

#rho inicia muito grande
rho = np.inf 

sim.startSimulation()

while rho > 0.05:
    robotPos = sim.getObjectPosition(robotHandle, -1)
    robotOri = sim.getObjectOrientation(robotHandle, -1)

    # config = (x,y,theta)
    robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])

    # erros 
    dx, dy, dth = qgoal - robotConfig

    # variaveis de estados atual
    rho = np.sqrt(dx**2 + dy**2)
    alpha = normalizeAngle(-robotConfig[2] + np.arctan2(dy,dx))
    beta = normalizeAngle(qgoal[2] - np.arctan2(dy,dx))

    kr = 4 / 20
    ka = 8 / 20
    kb = -1.5 / 20

    if abs(alpha) > np.pi/2:
        kr = -kr       
            
        # Se não ajustar a direção muda
        alpha = normalizeAngle(alpha-np.pi)
        beta = normalizeAngle(beta-np.pi)
        
    v = kr*rho
    w = ka*alpha + kb*beta  
    
    # Limit v,w to +/- max
    v = max(min(v, maxv), -maxv)
    w = max(min(w, maxw), -maxw) 

    wr = ((2.0*v) + (w*L))/(2.0*r)
    wl = ((2.0*v) - (w*L))/(2.0*r)

    sim.setJointTargetVelocity(rightMotor, wr)
    sim.setJointTargetVelocity(leftMotor, wl)

sim.setJointTargetVelocity(rightMotor, 0)
sim.setJointTargetVelocity(leftMotor, 0)

sim.stopSimulation()
        

In [ ]:
####################################################################################
#                                                                                  #
#                 LEMBRE-SE QUE A SIMULAÇÃO DEVE ESTAR EM EXECUÇÃO!                #
#                                                                                  #
####################################################################################

# Normalize angle to the range [-pi,pi)
def normalizeAngle(angle):
    return np.mod(angle+np.pi, 2*np.pi) - np.pi

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID!=-1:
    print ('Connected to remote API server')

    robotname = 'Pioneer_p3dx'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotname, sim.simx_opmode_oneshot_wait)    
    
    returnCode, robotLeftMotorHandle  = sim.simxGetObjectHandle(clientID, robotname + '_leftMotor', sim.simx_opmode_oneshot_wait)
    returnCode, robotRightMotorHandle = sim.simxGetObjectHandle(clientID, robotname + '_rightMotor', sim.simx_opmode_oneshot_wait)
        
    # Goal configuration (x, y, theta)
    qgoal = np.array([2, -2, np.deg2rad(90)])
    #qgoal = np.array([-2, -4, np.deg2rad(180)])
    
    # Frame que representa o Goal
    returnCode, goalFrame = sim.simxGetObjectHandle(clientID, 'Goal', sim.simx_opmode_oneshot_wait)     
    returnCode = sim.simxSetObjectPosition(clientID, goalFrame, -1, [qgoal[0], qgoal[1], 0], sim.simx_opmode_oneshot_wait)
    returnCode = sim.simxSetObjectOrientation(clientID, goalFrame, -1, [0, 0, qgoal[2]], sim.simx_opmode_oneshot_wait)    
    
    # Específico do robô
    L = 0.331
    r = 0.09751
    maxv = 1.0
    maxw = np.deg2rad(45)
    
    rho = np.inf
    while rho > .05:
        
        returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
        returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)        
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])
        
        dx, dy, dth = qgoal - robotConfig
        
        rho = np.sqrt(dx**2 + dy**2)
        alpha = normalizeAngle(-robotConfig[2] + np.arctan2(dy,dx))
        beta = normalizeAngle(qgoal[2] - np.arctan2(dy,dx))
        
        kr = 4 / 20
        ka = 8 / 20
        kb = -1.5 / 20
        
        # Alvo na parte de trás
        if abs(alpha) > np.pi/2:
            kr = -kr       
            
            # Se não ajustar a direção muda
            alpha = normalizeAngle(alpha-np.pi)
            beta = normalizeAngle(beta-np.pi)
        
        v = kr*rho
        w = ka*alpha + kb*beta
        
        # Limit v,w to +/- max
        v = max(min(v, maxv), -maxv)
        w = max(min(w, maxw), -maxw)        
        
        wr = ((2.0*v) + (w*L))/(2.0*r)
        wl = ((2.0*v) - (w*L))/(2.0*r)
        
        sim.simxSetJointTargetVelocity(clientID, robotRightMotorHandle, wr, sim.simx_opmode_oneshot_wait)
        sim.simxSetJointTargetVelocity(clientID, robotLeftMotorHandle, wl, sim.simx_opmode_oneshot_wait)

    sim.simxSetJointTargetVelocity(clientID, robotRightMotorHandle, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, robotLeftMotorHandle, 0, sim.simx_opmode_oneshot_wait)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
    
else:
    print ('Failed connecting to remote API server')
    
print ('Program ended')

In [ ]:
####################################################################################
#                                                                                  #
#                 LEMBRE-SE QUE A SIMULAÇÃO DEVE ESTAR EM EXECUÇÃO!                #
#                                                                                  #
####################################################################################

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID!=-1:
    print ('Connected to remote API server')

    robotname = 'Pioneer_p3dx'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotname, sim.simx_opmode_oneshot_wait)    
    
    returnCode, robotLeftMotorHandle  = sim.simxGetObjectHandle(clientID, robotname + '_leftMotor', sim.simx_opmode_oneshot_wait)
    returnCode, robotRightMotorHandle = sim.simxGetObjectHandle(clientID, robotname + '_rightMotor', sim.simx_opmode_oneshot_wait)
    
    returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
    returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
    
    # Goal position (x, y)
    pgoal = np.array([2, -2])
    pgoal = np.array([0, 0])
    
    # Frame que representa o Goal
    returnCode, goalFrame = sim.simxGetObjectHandle(clientID, 'Goal', sim.simx_opmode_oneshot_wait)     
    returnCode = sim.simxSetObjectPosition(clientID, goalFrame, -1, [pgoal[0], pgoal[1], 0], sim.simx_opmode_oneshot_wait)
    returnCode = sim.simxSetObjectOrientation(clientID, goalFrame, -1, [0, 0, 0], sim.simx_opmode_oneshot_wait)    
    
    
    # Específico do robô
    # https://www.generationrobots.com/media/Pioneer3DX-P3DX-RevA.pdf
    L = 0.381
    r = 0.0975
    maxv = 1.0
    maxw = np.deg2rad(45)
    
    rho = np.inf
    while rho > .05:
        
        returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
        returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)        
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])        
        
        dx, dy = pgoal - robotConfig[:2]
        
        # Apenas para interromper o loop
        rho = np.sqrt(dx**2 + dy**2)
                
        kr = 1
        kt = 2
        
        v = kr*(dx*np.cos(robotConfig[2]) + dy*np.sin(robotConfig[2]))
        w = kt*(np.arctan2(dy,dx) - robotConfig[2])
                
        # Limit v,w to +/- max
        v = max(min(v, maxv), -maxv)
        w = max(min(w, maxw), -maxw)        
        
        vr = ((2.0*v) + (w*L))/(2.0*r)
        vl = ((2.0*v) - (w*L))/(2.0*r)
        
        sim.simxSetJointTargetVelocity(clientID, robotRightMotorHandle, vr, sim.simx_opmode_oneshot_wait)
        sim.simxSetJointTargetVelocity(clientID, robotLeftMotorHandle, vl, sim.simx_opmode_oneshot_wait)

    sim.simxSetJointTargetVelocity(clientID, robotRightMotorHandle, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, robotLeftMotorHandle, 0, sim.simx_opmode_oneshot_wait)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
    
else:
    print ('Failed connecting to remote API server')
    
print ('Program ended')